In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import trapz
import os
from pathlib import Path

# === PARAMETERS ===
verified_bouts_file = r"C:\Users\labadmin\Documents\Uppsala analyses\clean_bout_verified.xlsx"
respirometry_dir = Path(r"D:\Respirometry\Bumblebee\flying")
BUFFER = 1  # optional time buffer (sec)

# === Load clean bouts ===
bouts_df = pd.read_excel(verified_bouts_file)
bouts_df = bouts_df[bouts_df["is_event_clean"] == "yes"].copy()
bouts_df["BeeID"] = bouts_df["BeeID"].str.strip().str.lower()

# === Function to find respirometry file by bee ID ===
def find_file(bee_id, directory):
    bee_id = bee_id.lower()
    for file in os.listdir(directory):
        if bee_id in file.lower():
            return directory / file
    return None

# === Extract segments ===
results = []

for idx, row in bouts_df.iterrows():
    bee = row["BeeID"]
    start = row["start_respi_s"]
    stop = row["stop_respi_s"]
    event = row["event_type"]

    file_path = find_file(bee, respirometry_dir)
    if not file_path:
        print(f"❌ No file found for {bee}")
        continue

    try:
        trace = pd.read_csv(file_path, sep="\t")
        trace["Seconds"] = pd.to_numeric(trace["Seconds"], errors="coerce")
        trace["CO2"] = pd.to_numeric(trace["CO2"], errors="coerce")
        trace["CO2_ppm"] = trace["CO2"] * 10000
        trace["H2O_pct"] = pd.to_numeric(trace["H2O"], errors="coerce")  # Percent H2O

        seg = trace[(trace["Seconds"] >= start - BUFFER) & (trace["Seconds"] <= stop + BUFFER)].copy()
        if seg.empty:
            print(f"⚠️ Empty segment for {bee} bout {idx}")
            continue

        seg["BeeID"] = bee
        seg["event_type"] = event
        seg["bout_index"] = idx
        results.append(seg)

    except Exception as e:
        print(f"❌ Error for {bee}: {e}")
        continue

# Combine and save
if results:
    full_df = pd.concat(results, ignore_index=True)
    full_df.to_csv("extracted_clean_bout_co2_h2o_data.csv", index=False)
    print("✅ CO₂ and H₂O data extracted and saved")
else:
    print("❌ No bouts extracted.")


✅ CO₂ and H₂O data extracted and saved


In [2]:
# Load data and metadata
df = pd.read_csv("extracted_clean_bout_co2_h2o_data.csv")
bee_meta = pd.read_excel("C:/Users/labadmin/Documents/Uppsala analyses/beeID laser file name + weight.xlsx", sheet_name="flying_bees")
bee_meta["BeeID"] = bee_meta["BeeID"].str.strip().str.lower()

# Merge in temperature
df["BeeID"] = df["BeeID"].str.strip().str.lower()
df = df.merge(bee_meta[["BeeID", "Temperature", "Average weight"]], on="BeeID", how="left")
df = df.rename(columns={"Average weight": "weight_g"})

# Constants
FLOW_RATE = 400  # mL/min
T_N = 23.32258   # Reference temperature for Q10
P_B = 760        # Barometric pressure (mmHg)

results = []

# Group by bee and bout
grouped = df.groupby(["BeeID", "bout_index"])

for (bee_id, bout_idx), group in grouped:
    group = group.sort_values("Seconds")
    t = group["Seconds"].values
    co2 = group["CO2_ppm"].values
    h2o_pct = group["H2O_pct"].values
    T_a = group["Temperature"].iloc[0]
    weight_g = group["weight_g"].iloc[0]
    event = group["event_type"].iloc[0]

    if pd.isna(T_a) or len(t) < 2:
        continue

    dt = t[-1] - t[0]
    if dt <= 0:
        continue

    q10 = 10 ** ((T_N - T_a) * (np.log10(2) / 10))
    area_co2_ppm_sec = trapz(co2, t)

    co2_rate_raw = (((area_co2_ppm_sec * q10 / 1e6 * FLOW_RATE) / 60) / dt * 3600 * 1e3)  # µL/h

    # Mean H2O over bout in ppm
    h2o_ppm_mean = np.nanmean(h2o_pct) * 10000
    p_h2o = (h2o_ppm_mean / 1e6) * P_B

    # STPD correction factor
    stpd_factor = ((P_B - p_h2o) / 760) * (273.15 / (273.15 + T_a))

    # Apply STPD correction to CO2 and O2
    co2_stpd = (co2_rate_raw / 1000) * stpd_factor  # mL/h

    results.append({
        "BeeID": bee_id,
        "bout_index": bout_idx,
        "event_type": event,
        "dt_sec": dt,
        "Temperature": T_a,
        "q10": q10,
        "H2O_ppm_mean": h2o_ppm_mean,
        "P_H2O": p_h2o,
        "STPD_factor": stpd_factor,
        "CO2_rate_mL_h_STPD": co2_stpd,
        "weight_g": weight_g
    })

# Save result
result_df = pd.DataFrame(results)
result_df.to_csv("metabolic_rates_stpd_corrected.csv", index=False)
print("✅ STPD-corrected metabolic rates saved to 'metabolic_rates_stpd_corrected.csv'")
result_df.head()


✅ STPD-corrected metabolic rates saved to 'metabolic_rates_stpd_corrected.csv'


,BeeID,bout_index,event_type,dt_sec,Temperature,q10,H2O_ppm_mean,P_H2O,STPD_factor,CO2_rate_mL_h_STPD,weight_g
0,green3,0,buzz,2,23.5,0.987777,-503.667767,-0.382788,0.921246,5.590133,0.181
1,green3,1,buzz,1,23.5,0.987777,-505.011500,-0.383809,0.921247,5.635173,0.181
2,green3,2,buzz,2,23.5,0.987777,-500.730267,-0.380555,0.921243,5.681205,0.181
3,green3,3,buzz,1,23.5,0.987777,-497.334450,-0.377974,0.921240,5.727241,0.181
4,green3,4,buzz,2,23.5,0.987777,-494.938900,-0.376154,0.921238,6.111753,0.181
